In [5]:
import contextlib
import io

f = io.StringIO()
with contextlib.redirect_stdout(f):
    import pandas as pd
    import numpy as np
    import cvxpy as cp
    import matplotlib.pyplot as plt

    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

In [6]:
df = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")

In [7]:
def spectral_regularization_model(_lambda,df):
    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']] #set of all known ratings
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    R=cp.Variable(shape=(num_users,num_anime)) #predicted rating matrix
    
    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx

    training_error=[]
    for ui in range(len(delta)):
        u=user_id_to_index[int(delta.at[ui,'u_id'])]
        i=anime_id_to_index[int(delta.at[ui,'a_id'])]
        training_error.append((R[u][i]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)


    obj=cp.Minimize(0.5*training_error_sum+ _lambda*cp.norm(R,"nuc"))
    prob=cp.Problem(obj)
    prob.solve()

    print("status:", prob.status)

    return np.matrix(R.value)

In [8]:
R = spectral_regularization_model(10000, df)


status: optimal
